In [28]:
from bs4 import BeautifulSoup
import json
import os
import requests

In [48]:
full_info = {
    "name" : [],
    "prepayment_amount":[],
    "body" : [],
    "issue_year" : [],
    "color" : [],
    "drive_unit" : [],
    "eng_capacity" : [],
    "mileage" : [],
    "condition" : [],
    "fuel_type" : [],
    "cleared" : [],
    "transmission" : [],
    "published_time" : [],
    "city" : [],
    "price" : [],
    "views" : [],
    "link" : [],
    "description" : []
}

In [3]:
full_info.keys()

dict_keys(['name', 'prepayment_amount', 'body', 'issue_year', 'color', 'drive_unit', 'eng_capacity', 'mileage', 'condition', 'fuel_type', 'cleared', 'transmission', 'published_time', 'city', 'price', 'views', 'link', 'description'])

In [49]:
# writing a car info to a hashmap
for i in range(1,51):
    prep_car_link = "somon_tj/pages/page"+str(i)+"/car_page"+str(i)+"/"
    car_list = list(os.walk("somon_tj/pages/page"+str(i)+"/car_page"+str(i)))[0][2:][0]
    with open("somon_tj/pages/page"+str(i)+"/car_page"+str(i)+".json","r",encoding='UTF-8') as jsonfile:
        car_hash = json.load(jsonfile)
    for car in car_list:
        x+=1
        with open(prep_car_link+car,"r",encoding="utf-8") as main_file:
            page = main_file.read()
            soup = BeautifulSoup(page,"lxml")
            #adding values to hash
            info = soup.find("ul",class_="chars-column").findAll("li",class_="")
            name_columns = ['body','issue_year', 'color', 'drive_unit', 'eng_capacity', 'mileage', 'condition', 'fuel_type', 'cleared', 'transmission']

            
            if info[0].find('span',class_="value-chars") is None:
                full_info['prepayment_amount'].append(0)
                for  index,column in enumerate(name_columns):
                    try :
                        full_info[column].append(info[index].find('a',class_="value-chars").text)
                    except:
                        full_info[column].append("Null")
            else:
                full_info['prepayment_amount'].append( info[0].find('span',class_="value-chars").text)
                for  index,column in enumerate(name_columns):
                    try:
                        full_info[column].append(info[index+1].find('a',class_="value-chars").text)
                    except:
                        full_info[column].append("Null")
            try:
                full_info['name'].append(car[:-5])
            except:
                full_info['name'].append("Null")
            try:
                full_info['description'].append(soup.find("div",class_="js-description").text.strip())
            except:
                full_info['description'].append("Null")
            try:
                full_info['price'].append(soup.find("div",class_="announcement-price__wrapper").text.strip().replace(' ','').replace('\n',' '))
            except:
                full_info['price'].append("Null")
            try:
                full_info['city'].append(soup.find("a",class_="announcement__location js-open-announcement-location").text.strip())
            except:
                full_info['city'].append('Null')
            try:
                full_info['views'].append(soup.find("span",class_="counter-views").text.strip().replace('Просмотров: ',''))
            except:
                full_info['views'].append('Null')
            try:
                full_info['published_time'].append(soup.find("span",class_="date-meta").text.replace('Опубликовано: ',''))
            except:
                full_info['published_time'].append("Null")
            try:
                full_info['link'].append(car_hash[car[:-5]])
            except:
                full_info['link'].append("Null")

In [125]:
import pandas as pd

In [155]:
df = pd.DataFrame(full_info)

In [156]:
# converting columns to another format of data
df['prepayment_amount'] = df['prepayment_amount'].apply(lambda x:int(str(x).replace(' c.','')))
df['issue_year'] = df['issue_year'].astype('int64') 
df['eng_capacity'] = df['eng_capacity'].apply(lambda x:float(x.replace(' л.','')))

In [159]:
# fixing bag
df1 = df[(df['mileage'] == 'С пробегом') | (df['mileage'] == 'Новая') | (df['mileage']=='Ретро') | (df['mileage']=='Аварийная')]
df1

,name,prepayment_amount,body,issue_year,color,drive_unit,eng_capacity,mileage,condition,fuel_type,cleared,transmission,published_time,city,price,views,link,description
9,"Daewoo Damas, 2005",12600,Седан,2005,Другой цвет,Передний,0.8,С пробегом,Газ,Да,Механика,Null,05.03.2023 17:45,Спитамен,42000c. торг,288,https://somon.tj/adv/9562983_daewoo-damas-2005/,100%
12,"Daewoo Nexia, 1996",0,Седан,1996,Белый,Полный,1.6,С пробегом,Бензин + газ,Да,Механика,Null,Сегодня 18:36,Пяндж,15000c.,33,https://somon.tj/adv/9579388_daewoo-nexia-1996/,нарх ахиршай
21,"Lexus RX series, 2007",40500,Кроссовер,2007,Чёрный,Полный,3.5,С пробегом,Бензин,Да,Автомат,Null,Сегодня 18:45,Душанбе,135000c. торг,19,https://somon.tj/adv/9579432_lexus-rx-series-2...,мошин дар холати нагз карор дорад савдо меша с...
32,"Opel Astra F, 1993",0,Седан,1993,Синий,Передний,1.6,С пробегом,Бензин + газ,Да,Механика,Null,Сегодня 18:44,Душанбе,14000c.,138,https://somon.tj/adv/9579428_opel-astra-f-1993/,Мошин на ходу камтар кор дора дуд мекна па...
37,"Opel Astra G, 1999",17400,Универсал,1999,Голубой,Передний,1.6,Новая,Бензин,Да,Механика,Null,Сегодня 18:42,Душанбе,58000c. торг,24,https://somon.tj/adv/9579423_opel-astra-g-1999/,мошин дар холати хуб карор дорад хучат дора
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2988,"Opel Astra F, 1995",9000,Седан,1995,Красный,Передний,1.6,С пробегом,Бензин + газ,Да,Механика,Null,Вчера 09:20,Душанбе,30000c. торг,278,https://somon.tj/adv/9048535_opel-astra-f-1995/,Продается Опель Седан \nМашина в лучшем состоя...
2999,"Opel Zafira, 2006",17400,Кроссовер,2006,Серебристый,Передний,1.6,С пробегом,Бензин + газ,Да,Механика,Null,Вчера 09:19,Душанбе,58000c.,173,https://somon.tj/adv/9457318_opel-zafira-2006/,СРОЧНО МОШИН КАМАКАК ГАШТАГИ АХИРИ НАРХШАЙ....
3002,"Toyota Camry, 2008",33000,Седан,2008,Синий,Передний,2.4,С пробегом,Гибрид,Да,Автомат,Null,Вчера 09:18,Душанбе,110000c. торг,72,https://somon.tj/adv/9557251_toyota-camry-2008/,Мошин да холати хуб нав омадаги растаможка кад...
3005,"Toyota Corolla, 2004",20400,Универсал,2004,Белый,Передний,1.4,С пробегом,Бензин + газ,Да,Механика,Null,Вчера 09:22,Душанбе,68000c. торг,131,https://somon.tj/adv/9496087_toyota-corolla-2004/,мошин дар холати хуби техники карор дорад. 4 б...


In [160]:
x = (df['mileage'] == 'С пробегом') | (df['mileage'] == 'Новая') | (df['mileage']=='Ретро') | (df['mileage']=='Аварийная')
df.loc[x,'condition'] = df1[x]['mileage']
df.loc[x,'fuel_type'] = df1[x]['condition']
df.loc[x,'cleared'] = df1[x]['fuel_type']
df.loc[x,'transmission'] = df1[x]['cleared']
df.loc[x,'mileage'] = "0"

C:\Users\abuzu\AppData\Local\Temp\ipykernel_3440\4127641571.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df.loc[x,'condition'] = df1[x]['mileage']
C:\Users\abuzu\AppData\Local\Temp\ipykernel_3440\4127641571.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df.loc[x,'fuel_type'] = df1[x]['condition']
C:\Users\abuzu\AppData\Local\Temp\ipykernel_3440\4127641571.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df.loc[x,'cleared'] = df1[x]['fuel_type']
C:\Users\abuzu\AppData\Local\Temp\ipykernel_3440\4127641571.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df.loc[x,'transmission'] = df1[x]['cleared']


In [161]:
df['mileage'] = df['mileage'].apply(lambda x:str(x).replace('км','')).astype('int64')

In [162]:
df['published_time'] = df['published_time'].apply(lambda x:x.replace('Сегодня','08.03.2023'))
df['published_time'] = df['published_time'].apply(lambda x:x.replace('Вчера','07.03.2023'))
df['published_time'] = df['published_time'].apply(lambda x:x+':00')

In [163]:
df['published_time'] = pd.to_datetime(df['published_time'], format='%d.%m.%Y %H:%M:%S')

In [164]:
df['price'] = df['price'].apply(lambda x:x.replace('торг','').split('  ')[0].replace('c.','')).astype('int64')

In [165]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3017 entries, 0 to 3016
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   name               3017 non-null   object        
 1   prepayment_amount  3017 non-null   int64         
 2   body               3017 non-null   object        
 3   issue_year         3017 non-null   int64         
 4   color              3017 non-null   object        
 5   drive_unit         3017 non-null   object        
 6   eng_capacity       3017 non-null   float64       
 7   mileage            3017 non-null   int64         
 8   condition          3017 non-null   object        
 9   fuel_type          3017 non-null   object        
 10  cleared            3017 non-null   object        
 11  transmission       3017 non-null   object        
 12  published_time     3017 non-null   datetime64[ns]
 13  city               3017 non-null   object        
 14  price   

In [172]:
# creating new columns
df['year'] = df['name'].apply(lambda x:x.split(',')[1]).astype('int64')
df['car_name'] = df['name'].apply(lambda x:x.split(',')[0].split()[0])
df['model'] = df['name'].apply(lambda x:' '.join(x.split(',')[0].split()[1:]))

In [186]:
df

,name,prepayment_amount,body,issue_year,color,drive_unit,eng_capacity,mileage,condition,fuel_type,...,transmission,published_time,city,price,views,link,description,year,car_name,model
0,"BMW 7 series, 2018",0,Седан,2018,Чёрный,Полный,5.0,77000,С пробегом,Бензин,...,Автомат,2023-03-08 18:32:00,Душанбе,550000,2022,https://somon.tj/adv/9339378_bmw-7-series-2018/,Мошин Корейский состоянии зур.Полный фулл комп...,2018,BMW,7 series
1,"BMW Gran Turismo, 2018",0,Седан,2018,Чёрный,Задний,3.0,100,С пробегом,Дизель,...,Автомат,2023-03-08 18:56:00,Душанбе,470000,1148,https://somon.tj/adv/9108321_bmw-gran-turismo-...,Bmw 630M GT в отличном состоянии,2018,BMW,Gran Turismo
2,"BMW X5, 2011",64500,Внедорожник,2011,Чёрный,Полный,3.0,145000,С пробегом,Дизель,...,Автомат,2023-03-08 18:46:00,Душанбе,215000,38,https://somon.tj/adv/9579439_bmw-x5-2011/,Мошин дар холати хубай\n10 мох танировка хучат...,2011,BMW,X5
3,"BYD Yuan Plus, 2022",0,Кроссовер,2022,Белый,Передний,1.6,300,Новая,Электричество,...,Автомат,2023-03-08 18:41:00,Душанбе,275000,18,https://somon.tj/adv/9579419_byd-yuan-plus-2022/,электромобиль,2022,BYD,Yuan Plus
4,"BYD, 2022",0,Седан,2022,Белый,Передний,1.6,300,Новая,Электричество,...,Автомат,2023-03-08 18:37:00,Душанбе,275000,13,https://somon.tj/adv/9579391_byd-drugie-modeli...,электромобиль,2022,BYD,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3012,"Toyota Voxy, 2009",24000,Фургон,2009,Серебристый,Передний,2.0,137000,С пробегом,Бензин,...,Автомат,2023-03-07 19:30:00,Душанбе,80000,919,https://somon.tj/adv/9518242_toyota-voxy-2009/,2009 Voxy. Мошин нав даромад. Дар Душанбе. Дар...,2009,Toyota,Voxy
3013,"Volkswagen Golf, 2000",11700,Хэтчбек,2000,Серебристый,Передний,1.6,9696,С пробегом,Бензин + газ,...,Механика,2023-03-07 09:34:00,Худжанд,39000,124,https://somon.tj/adv/9502671_volkswagen-golf-2...,4электра гидрвлик абс \nдиск 15 кодисонер 2 ай...,2000,Volkswagen,Golf
3014,"Volkswagen Golf, 2012",33000,Седан,2012,Чёрный,Передний,2.0,111111,С пробегом,Дизель,...,Автомат,2023-03-07 09:16:00,Худжанд,110000,137,https://somon.tj/adv/9451530_volkswagen-golf-2...,"Мошин дар холати хуб, любойхэ проверка,вагон",2012,Volkswagen,Golf
3015,"Volkswagen Passat, 2009",18900,Седан,2009,Серебристый,Передний,2.0,123456,С пробегом,Бензин,...,Автомат,2023-03-07 09:09:00,Душанбе,63000,92,https://somon.tj/adv/9507547_volkswagen-passat...,Мотор. 2.0 турбо \n4 балони пачка\nЗадний кам...,2009,Volkswagen,Passat


In [187]:
# writing csv file from DataFrame
df.to_csv('cleared_data.csv')